<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/AccessDiagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Prerequisites
!pip install --upgrade --force pydantic git+https://github.com/fair-research/bdbag git+https://github.com/informatics-isi-edu/deriva-py git+https://github.com/informatics-isi-edu/eye-ai-tools git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml
from eye_ai import EyeAI


In [ ]:
# login to DERIVA via Globus Auth
DEBUG_MODE = False #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'
!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}

Make a connection to the Eye AI catalog and get a reference to the main schema: eye-ai

In [ ]:
import pandas as pd

from deriva.core import ErmrestCatalog, get_credential
protocol = 'https'
catalog_number = 'eye-ai'
credential = get_credential(DEFAULT_SERVER)
catalog = ErmrestCatalog(protocol, DEFAULT_SERVER, catalog_number, credential)
# Get the path builder interface for this catalog
pb = catalog.getPathBuilder()
eye_ai = pb.schemas['eye-ai']

# Initialize the tool
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_id = 'eye-ai')

In [ ]:
# @title Dataset Tags

pd.DataFrame(eye_ai.Dataset.path.attributes(eye_ai.Dataset.RID, eye_ai.Dataset.Description))

In [ ]:
# @title Diagnosis Tags
pd.DataFrame(eye_ai.Diagnosis_Tag.path.attributes(
    eye_ai.Diagnosis_Tag.RID,
    eye_ai.Diagnosis_Tag.Name,
    eye_ai.Diagnosis_Tag.Description))


In [ ]:
# @title Configure
diagnosis = eye_ai.Diagnosis  # All of the diagnosis in the catalog
dataset_rid = "2-277M" # @param {type:"string"}
diagnosis_tag_rid = "2-35RM" # @param {type:"string"}
compare_value = "Cup/Disk_Ratio" #@param {type:"string"}["Diagnosis", "Image_Quality", "Cup/Disk_Ratio"]
initial_diagnosis_tag_rid = "C1T4"
angle_two_rid = '2SK6'

dataset = eye_ai.Dataset
ds = dataset.path.filter(dataset.RID==dataset_rid).attributes(dataset.RID, dataset.Description).fetch()[0]
print(f"Dataset tag (RID:{dataset_rid}):\n\tdescription: {ds['Description']}")

diagnosis_tag = eye_ai.Diagnosis_Tag
diagnosis_info = diagnosis_tag.path.filter(diagnosis_tag.RID==diagnosis_tag_rid).attributes(diagnosis_tag.RID, diagnosis_tag.Name, diagnosis_tag.Description).fetch()[0]
print(f"Diagnosis tag (RID:{diagnosis_tag_rid}):\n\tname {diagnosis_info['Name']} \n\tdescription: {diagnosis_info['Description']}")

print(f"In the wide table, {compare_value} will be compared among graders and the initial diagnosis.")

In [ ]:
# @title Example of Graded Test Dataset

Graded_test_initial_diag = EA.image_tall(dataset_rid, initial_diagnosis_tag_rid)
Graded_test_grader_diag = EA.image_tall(dataset_rid, diagnosis_tag_rid)
# Graded_test_gold = EA.image_tall(dataset_rid, "2-5GXP")
long, wide = EA.reshape_table([Graded_test_initial_diag, Graded_test_grader_diag], compare_value)


# long.to_csv("GradedTest_diag_long.csv")
# wide.to_csv("GradedTest_diag_wide.csv")


In [ ]:
long

In [ ]:
wide